# Q1 who is more influential??

Choice is the human judgement (A>B then 1), meaning if 1, A has higher influence power according to humans

**HOWEVER, how does "human judgement" reflect the real power of influece**

In [5]:
# create prediction models and choose the best model. For Q2, models that can extract feature importance are recommneded

# for details of the data, refer to the Data Description in the "part_1_eda_features_eng.ipynb"

# maybe we should scale or standardize diff_following, diff_listed, diff_mentions or do something about the outlies. 

In [6]:
import pandas as pd

df = pd.read_csv('train_v2.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5368 entries, 0 to 5499
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   diff_following_count          5368 non-null   int64  
 1   diff_listed_count             5368 non-null   int64  
 2   diff_mentions_received        5368 non-null   float64
 3   diff_retweets_received        5368 non-null   float64
 4   diff_mentions_sent            5368 non-null   float64
 5   diff_retweets_sent            5368 non-null   float64
 6   diff_posts                    5368 non-null   float64
 7   diff_network_feature_1        5368 non-null   int64  
 8   diff_network_feature_2        5368 non-null   float64
 9   diff_network_feature_3        5368 non-null   float64
 10  ratio_follower_count_A/B      5368 non-null   float64
 11  ratio_following_count_A/B     5368 non-null   float64
 12  ratio_mentions_received_A/B   5368 non-null   float64
 13  rat

# Q2 which factors are the best predictors of influence?

# Q3 Report the confusion matrix of your “best” model

# Q4 Are there any surprises here? How can a business use your model/results?

> what exactly are network features 1,2,3? 